##### Copyright 2019 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=y_UVSRtBBsJk).

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

# Image Classification using Swift for TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/13lBsht3Wa4GjKKkA47JCrd54XikhNX2E"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="Link to be updated"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub link to be updated accordingly</a>
  </td>
</table>

In this Colab you will classify images of flowers. We'll build an image classifier using `Layer` and load data by creating training and validation tensors of images as well as their corresponding labels.

# Importing packages

Let's start by importing required packages:

*   glob — to read files and directory structure.
*   numpy — for some matrix math outside of TensorFlow.
*   matplotlib.pyplot — to plot the graph and display images in our training and validation data.
*  PIL — to view images.

In [0]:
import TensorFlow
import Foundation
import Python

%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")
let np = Python.import("numpy")  // Make numpy available using np.
let subprocess = Python.import("subprocess")
let plt = Python.import("matplotlib.pyplot")
let os = Python.import("os")
let glob = Python.import("glob")
let pil = Python.import("PIL")

// Import packages.

In order to build our image classifier, we can begin by downloading the flowers dataset.  

In [0]:
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String {
        let (task, pipe) = (Process(), Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments, task.standardOutput) = (args, pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

In [0]:
print("/bin/ls".shell("-lh"))

In order to build our image classifier, we can begin by downloading the flowers dataset. We first need to download the archive version of the dataset and after the download, we unzip it.

In [0]:
let command = "wget -nv -O- https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz | tar xzf - -C ."
subprocess.call(command, shell: true)

The  dataset we downloaded contains images of 5 types of flowers:

1. Rose
2. Daisy
3. Dandelion
4. Sunflowers
5. Tulips

So, let's create the labels for these 5 classes: 

In [0]:
let classNames = ["roses", "daisy", "dandelion", "sunflowers", "tulips"]

Also, the dataset we have downloaded has following directory structure:

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower_photos</b>
|__ <b>diasy</b>
|__ <b>dandelion</b>
|__ <b>roses</b>
|__ <b>sunflowers</b>
|__ <b>tulips</b>
</pre>

In [0]:
let filelist = try FileManager.default.contentsOfDirectory(atPath: "./flower_photos/daisy")
for filename in filelist[0..<10] {
    print(filename)
}

In [0]:
let totalImages = glob.glob("flower_photos/*/**.jpg")
let daisyList = glob.glob("flower_photos/daisy/*.jpg")
let dandelionList = glob.glob("flower_photos/dandelion/*.jpg")
let rosesList = glob.glob("flower_photos/roses/*.jpg")
let sunflowersList = glob.glob("flower_photos/sunflowers/*.jpg")
let tulipsList = glob.glob("flower_photos/tulips/*.jpg")

print("Total Images \(totalImages.count)")
print("roses \(rosesList.count)")
print("sunflowers \(sunflowersList.count)")
print("daisy \(daisyList.count)")
print("tulips \(tulipsList.count)")
print("dandelion \(dandelionList.count)")

We'll now assign variables with the proper file path for the training and validation sets.

As you can see there are no folders containing training and validation data. Therefore, we will have to create our own training and validation set. Let's write some code that will do this.


The code below creates a `trainList` and a `testList` list. It then copies the image paths from the original folders to these new lists such that 80% of the images go to the training set and 20% of the images go into the validation set.

In [0]:
let numTotalImages = [641, 699, 633, 799, 898]
let numTrainImages = [512, 559, 506, 639, 718]  // Number of Train Images = 0.8 * Number of Total Images.
var trainList = Python.list()
var testList = Python.list()
var i = 0, numImagesDone = 0

for path in totalImages {
    if numImagesDone == numTotalImages[i] {
        i += 1
        numImagesDone = 0
    }
    if numImagesDone >= numTrainImages[i] {
        testList.append(path)
        numImagesDone += 1
    } else {
        trainList.append(path)
        numImagesDone += 1
    }
}

for i in 0 ..< 5 {
    np.random.shuffle(trainList)
    np.random.shuffle(testList)
}

print("Total Images \(totalImages.count)")
print("Train Images \(trainList.count)")
print("Test Images \(testList.count)")

### Visualizing Training images

We can visualize our training images by creating functions to plot images through their paths or tensors, and then plotting a few of them.

In [0]:
func plotImages(_ image: Tensor<Float>) {
    let numpyImage = image.makeNumpyArray().reshape(150, 150, 3)
    plt.imshow(numpyImage)
    plt.show()
}

func plotImages(fromPath path: String) {
    let img = pil.Image.open(path)
    let image = np.array(img) * (1.0 / 255)
    plt.imshow(image)
    plt.show()
}

## TODO: Apply Data Augmentation

Overfitting often occurs when we have a small number of training examples. One way to fix this problem is to augment our dataset so that it has sufficient number and variety of training examples. Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples through random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This exposes the model to more aspects of the data, allowing it to generalize better.

In Swift for TensorFlow we can implement this using the different Python libraries through Python interoperability. WWe can simply apply different transformations we would want to our dataset images and they will be applied during our training process.

We perform the following image augmentation techniques randomly on an image:

* Rotating the image
* Transposing
* Flipping the image horizontally
* Adding noise

In [0]:
func augmentedImage(path: String) -> Tensor<Float> {
    
    var img = pil.Image.open(path)
    
    // Rotate 45 degrees.
    if random.random() < 0.5 {
        img =  
    }
    
    // Transpose.
    if random.random() < 0.5 {
        img = 
    }
    
    // Flip.
    if random.random() < 0.5 {
        img = 
    }
    
    var image = np.array(img, dtype: np.float32) * (1.0 / 255)
    
    // Add noise.
    if random.random() < 0.5 {
        image =  
    }
    
    image = np.array(image, dtype: np.float32)
    
    let imageTensor = Tensor<Float>(numpy: image)!

    return imageTensor
}

# Data Preparation 

Images must be formatted into appropriately pre-processed floating point tensors before being fed into the network. The steps involved in preparing these images are:

1. Read images from the disk.
2. Decode the contents of these images into their RGB bytes.
3. Convert them into floating point tensors.
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, to better match the range expected by the initial neural network weights.
5. Apply or don't apply image augmentation techniques, based on the type of data.

We have done this in the following code.

The `resizedImage(fromPath:augmented:)` function takes 2 inputs:
1. Image path as input.
2. Whether or not to apply image augmentations.

The `images(fromList:imageCount:augmented:)` function takes 3 inputs :

1. List of paths.

2. Number of tensors to be produced in the output tensor as input.

3. Whether or not to apply image augmentations.

In [0]:
func resizedImage(fromPath: String, augmented: Bool) -> (Tensor<Float>, Int32) {
    var img = pil.Image.open(fromPath)
    var image = np.array(img, dtype: np.float32) * (1.0 / 255)
    var imageTensor = Tensor<Float>(numpy: image)!
    
    // Process image through Image Augmentation Techniques depending on the value of `ImageAugmentation: Bool`.
    
    var label: Int32 = 0
    
    // Decide label according to the path of image.
    
    return (imageTensor, label)
}

func images(fromList: PythonObject, imageCount: Int, augmented: Bool) -> (image: Tensor<Float>, label: Tensor<Int32>) {
    let batchFiles = fromList[0..<imageCount]
    var labels: [Int32] = []
    var x: Tensor<Float>
    var y: Tensor<Int32>

    // Create Tensor Dataset using `resizedImage(fromPath:augmented:)` function.
    
    return (x, y)
}

After defining our generators for images and labels, we will load those images and labels in tensor arrays, thereby creating our `testTensors`.

In [0]:
let testTensors = 
let testImageTensors = testTensors.0
let testLabelTensors = testTensors.1
print(testImageTensors.shape)
print(testLabelTensors.shape)

# Model Creation

## TODO: Create the CNN

In the cell below, create a convolutional neural network that consists of 3 convolution blocks. Each convolutional block contains a `Conv2D` layer followed by a max pool layer. The first convolutional block should have 16 filters, the second one should have 32 filters, and the third one should have 64 filters. All convolutional filters should be 3 x 3. All max pool layers should have a `poolSize` of `(2, 2)`.

After the 3 convolutional blocks you should have a flatten layer followed by a fully connected layer with 512 units. The CNN should output class probabilities based on 5 classes which is done by the **softmax** activation function. All other layers should use a **relu** activation function. You should also add Dropout layers with a probability of 20%, where appropriate. 

In [0]:
struct Classifier: Layer {
    typealias Input = Tensor<Float>
    typealias Output = Tensor<Float>
    
    @differentiable
    public func callAsFunction(_ input: Input) -> Output {
        
    }
}

## TODO: Compile the model

In the cell below, compile your model using the adam optimizer and sparse cross entropy function as a loss function. We would also like to look at the training and validation accuracy on each epoch as we train our network, so make sure you also pass the metrics argument.

In [0]:
let tensor = Tensor<Float>(zeros:[1, 150, 150, 3])
var classifier = 
var optimizer = 
classifier(tensor).shape

In the cell below, create a batchSize of 100 images and epochCount of 80.

In [0]:
let epochCount = 
let batchSize = 

// Extract a batch of size batchSize.
func minibatch<Scalar>(in x: Tensor<Scalar>, at index: Int) -> Tensor<Scalar> {
    
}

In [0]:
var trainingAccuracy: [Float] = []
var validationAccuracy: [Float] = []
var trainingLoss: [Float] = []
var validationLoss: [Float] = []
var epochsRange: [Int] = []

## TODO: Train the model

In the cell below, train your model using the ImageAugmentation function. We need to apply the random image augmentations every time an image is accessed, so we recreate `trainTensors` in every epoch. Train the model for 80 epochs and make sure you use the proper parameters.



In [0]:
print("Beginning training...")

struct Statistics {
    var correctGuessCount: Int = 0
    var totalGuessCount: Int = 0
    var totalLoss: Float = 0
}

// The training loop.
for epoch in 1...epochCount {
    
}

## TODO: Visualizing results of the training

In the cell below, plot the training and validation accuracy/loss graphs.

In [0]:
plt.figure(figsize: [12, 8])
plt.subplot(1, 2, 1)
plt.xlabel("")
plt.ylabel("Training Accuracy(l) vs Validation Accuracy(o)")
plt.plot()
plt.plot()
var loc = "lower right"
plt.legend()
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.xlabel("")
plt.ylabel("Training Loss(u) vs Validation Loss(p)")
plt.plot()
plt.plot()
loc = "upper right"
plt.legend()
plt.title("Training and Validation Loss")
plt.show()

# TODO: Experiment with Different Parameters

So far you've created a CNN with 3 convolutional layers and followed by a fully connected layer with 512 units. In the cells below create a new CNN with a different architecture. Feel free to experiement by changing as many parameters as you like. For example, you can add more convolutional layers, or more fully connected layers. You can also experiement with different filter sizes in your convolutional layers, different number of units in your fully connected layers, different dropout rates, etc... You can also experiment by performing image aumentation with more image transformations that we have seen so far. For example, you can add shear transformations, or you can vary the brightness of the images, etc... Experiment as much as you can and compare the accuracy of your various models. Which parameters give you the best result?